In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import math

from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import cv2 as cv
from PIL import Image

import multiprocessing
from multiprocessing.pool import ThreadPool

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelBinarizer

import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

import cv2

def preProcessImageObsolete(image, cutoff=127, maxContours=5):
    image = np.uint8(image)
    im = np.uint8(image)
    red, thresh = cv2.threshold(im, cutoff, 255, 0)
    im2, contours, hierarchy= cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.uint8(np.ones(im.shape))
    largest_contours = sorted(contours, key=cv2.contourArea)
    for ind, contour in enumerate(largest_contours[maxContours:]):
        mask = cv2.drawContours(mask, [largest_contours[ind]], -1, 0, -1)
        
    filteredImage = cv2.bitwise_and(thresh, thresh, mask=mask)
    ## plt.imshow(filteredImage)
    return filteredImage

def preprocess(image, cutoff=127, maxContours=5):
    image = np.uint8(image)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image,connectivity = 4)
    sizes = stats[:,-1]
    max_label = 1
    max_size = sizes[1]
    for i in range(2,nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]
    img = np.zeros(output.shape)
    img[output == max_label] = 255
    return img

def preProcessImage(image, cutoff=127, maxContours=10):
    image = np.uint8(image)
    im = np.uint8(image)
    red, thresh = cv2.threshold(im, cutoff, 255, 0)
    im2, contours, hierarchy= cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros(im.shape, np.uint8)
    largest_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    for ind, contour in enumerate(largest_contours[:5]):
        x, y, w, h = cv2.boundingRect(contour)
        mask[y:y+h, x:x+w] = 255    
    filteredImage = cv2.bitwise_and(thresh, thresh, mask=mask)
    #plt.imshow(filteredImage)
    #plt.figure()
    #plt.imshow(thresh)
    #plt.figure()
    #plt.imshow(mask)
    return filteredImage.reshape((image.shape))

data = np.load("dataset/train_images.npy",encoding='bytes')

x = []
for image in data:
    image = image[1].reshape(100,100)
    image = preprocess(image)
    x.append(image)

x_pre = []
for image in data:
    image = image[1].reshape(100,100)
    x_pre.append(image)



labels = pd.read_csv("dataset/train_labels.csv")
y = []
for i in range(len(labels)):
    label = labels['Category'][i]
    y.append(label)


x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size = 0.1, random_state=30)

x_train_backup = x_train
x_validation_backup = x_validation

x_train = np.array(x_train).reshape(len(x_train), 100, 100, 1).astype('float32') / 255
x_validation = np.array(x_validation).reshape(len(x_validation), 100, 100, 1).astype('float32') / 255

encoder = LabelBinarizer()

y_train = encoder.fit_transform(y_train)
y_validation = encoder.fit_transform(y_validation)

y_train_decoded = encoder.inverse_transform(y_train)
y_validation_decoded = encoder.inverse_transform(y_train)


##Setup CNN
model = Sequential()
model.add(Conv2D(filters=36,
                 kernel_size=(3, 3),
                 padding='same',
                 input_shape=(100, 100, 1),
                 activation='relu'))
model.add(Conv2D(filters=36,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=36,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'))
model.add(Conv2D(filters=36,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(units=512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(31, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

##Fit CNN model
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images
datagen.fit(x_train)




learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)




batch_size = 50
epochs = 100
train_history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    epochs=epochs, validation_data=(x_validation, y_validation),
                                    verbose=2, steps_per_epoch=x_train.shape[0] // batch_size
                                    , callbacks=[learning_rate_reduction])


plt.plot(train_history.history['acc'])
plt.plot(train_history.history['val_acc'])
epoch_num = len(train_history.epoch)
final_epoch_train_acc = train_history.history['acc'][epoch_num - 1]
final_epoch_validation_acc = train_history.history['val_acc'][epoch_num - 1]
plt.text(epoch_num, final_epoch_train_acc, 'train = {:.3f}'.format(final_epoch_train_acc))
plt.text(epoch_num, final_epoch_validation_acc-0.01, 'valid = {:.3f}'.format(final_epoch_validation_acc))
plt.title('Train History')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.xlim(xmax=epoch_num+1)
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


data_test = np.load("dataset/test_images.npy",encoding='bytes')

x_test = []
for image in data_test:
    image = image[1].reshape(100,100)
    image = preprocess(image)
    x_test.append(image)

x_test = np.array(x_test).reshape(len(x_test), 100, 100, 1).astype('float32') / 255

prediction = model.predict_classes(x_test)
df = pd.DataFrame(prediction)
df.index += 1
df.index.name = 'Id'
df.columns = ['Category']
df.to_csv('cnn.csv', header=True)

word_class = []
for i in range(len(df)):
    c = encoder.classes_[df['Category'][i+1]]
    word_class.append(c)

word_class = np.array(word_class)
word_class = pd.DataFrame(word_class)
word_class.columns = ['Category']
word_class.to_csv('cnn.csv', header=True)


Using TensorFlow backend.


Epoch 1/100
 - 9s - loss: 3.2382 - acc: 0.1020 - val_loss: 3.2485 - val_acc: 0.1390
Epoch 2/100
 - 7s - loss: 2.7326 - acc: 0.2057 - val_loss: 2.5533 - val_acc: 0.2200
Epoch 3/100
 - 8s - loss: 2.4716 - acc: 0.2598 - val_loss: 2.3514 - val_acc: 0.3050
Epoch 4/100
 - 7s - loss: 2.3558 - acc: 0.2997 - val_loss: 2.4110 - val_acc: 0.3010
Epoch 5/100
 - 8s - loss: 2.2365 - acc: 0.3404 - val_loss: 2.2813 - val_acc: 0.3290
Epoch 6/100
 - 7s - loss: 2.1523 - acc: 0.3561 - val_loss: 2.1495 - val_acc: 0.3570
Epoch 7/100
 - 7s - loss: 2.0617 - acc: 0.3966 - val_loss: 2.0789 - val_acc: 0.3860
Epoch 8/100
 - 7s - loss: 1.9851 - acc: 0.4187 - val_loss: 2.0585 - val_acc: 0.4170
Epoch 9/100
 - 7s - loss: 1.9383 - acc: 0.4249 - val_loss: 2.0866 - val_acc: 0.4080
Epoch 10/100
 - 7s - loss: 1.8735 - acc: 0.4486 - val_loss: 1.9230 - val_acc: 0.4410
Epoch 11/100
 - 7s - loss: 1.8350 - acc: 0.4606 - val_loss: 1.9354 - val_acc: 0.4510
Epoch 12/100
 - 7s - loss: 1.7903 - acc: 0.4746 - val_loss: 1.8366 - val_a

 - 7s - loss: 1.1210 - acc: 0.6604 - val_loss: 1.3999 - val_acc: 0.5910
Epoch 92/100
 - 7s - loss: 1.1131 - acc: 0.6667 - val_loss: 1.3955 - val_acc: 0.5910
Epoch 93/100
 - 7s - loss: 1.1182 - acc: 0.6612 - val_loss: 1.3915 - val_acc: 0.5930
Epoch 94/100
 - 7s - loss: 1.1237 - acc: 0.6631 - val_loss: 1.3941 - val_acc: 0.5920
Epoch 95/100
 - 7s - loss: 1.1271 - acc: 0.6533 - val_loss: 1.3946 - val_acc: 0.5960
Epoch 96/100
 - 8s - loss: 1.1158 - acc: 0.6658 - val_loss: 1.3913 - val_acc: 0.5930
Epoch 97/100
 - 7s - loss: 1.1166 - acc: 0.6621 - val_loss: 1.3916 - val_acc: 0.5980
Epoch 98/100
 - 7s - loss: 1.1270 - acc: 0.6644 - val_loss: 1.4015 - val_acc: 0.5900
Epoch 99/100
 - 7s - loss: 1.1035 - acc: 0.6653 - val_loss: 1.3926 - val_acc: 0.5910
Epoch 100/100
 - 7s - loss: 1.1229 - acc: 0.6613 - val_loss: 1.3946 - val_acc: 0.5920


C:\Users\Eric Vuong\Anaconda3\envs\tensorflowgpu\lib\site-packages\matplotlib\axes\_base.py:3122: MatplotlibDeprecationWarning: 
The `xmax` argument was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use `right` instead.
  alternative='`right`', obj_type='argument')


<Figure size 640x480 with 1 Axes>

In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)